In [1]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os

C:\Users\dleir\miniconda3\envs\tfm\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('poems_cleaned.csv')
df.dropna(axis=0, inplace=True)
#test_set = df.sample(n = 200)
#df = df.loc[~df.index.isin(test_set.index)]
test_set = df.copy()

In [3]:
test_set = test_set.reset_index()
df = df.reset_index()

In [4]:
#test_set['True_end_poems'] = test_set['poem'].str.split().str[-20:].apply(' '.join)
#test_set['poem'] = test_set['poem'].str.split().str[:-20].apply(' '.join)

In [5]:
class SpanishPoems(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="DeepESP/gpt2-spanish", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.poems = []

        for row in df['poem']:
          self.poems.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))               
        if truncate:
            self.poems = self.poems[:20000]
        self.poems_count = len(self.poems)
        
    def __len__(self):
        return self.poems_count

    def __getitem__(self, item):
        return self.poems[item]
    
dataset = SpanishPoems(df['poem'], truncate=True, gpt2_type="DeepESP/gpt2-spanish") 

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('DeepESP/gpt2-spanish')
model = GPT2LMHeadModel.from_pretrained('DeepESP/gpt2-spanish')

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [7]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir="./spanish_poems_model", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [8]:
model = train(dataset, model, tokenizer)

C:\Users\dleir\miniconda3\envs\tfm\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


20000it [15:34, 21.40it/s]


Training epoch 1
tensor(0.8953, device='cuda:0', grad_fn=<NllLossBackward0>)


20000it [15:37, 21.33it/s]


...

20000it [15:38, 21.30it/s]


Training epoch 3
tensor(1.1098, device='cuda:0', grad_fn=<NllLossBackward0>)


20000it [15:47, 21.11it/s]


Training epoch 4
tensor(1.6601, device='cuda:0', grad_fn=<NllLossBackward0>)


20000it [16:25, 20.29it/s]


In [22]:
torch.save(model, "spanish_poems_model_v1.0.pt")

In [10]:
prueba = """
No sé, dicen que es invierno
y que fuera está nevando.
Pero aquí sigue siendo verano.

Tú que te coronaste en Cotos sin necesidad de vestido,
o de trampas en papel de efecto retardado,
querías saber lo que me hizo el sol en el Atlántico
pero tú te quemaste la espalda
"""

In [11]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

In [12]:
def text_generation(text, temperature=1., top_p=0.8, entry_length=30):
  x = generate(model.to('cpu'), tokenizer, text, entry_count=1, temperature=temperature, top_p=top_p, entry_length=entry_length)
  print(x[0])

In [13]:
text_generation(prueba, temperature=1., top_p=0.8, entry_length=150)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [02:49<00:00, 169.43s/it]


No sé, dicen que es invierno
y que fuera está nevando.
Pero aquí sigue siendo verano.

Tú que te coronaste en Cotos sin necesidad de vestido,
o de trampas en papel de efecto retardado,
querías saber lo que me hizo el sol en el Atlántico
pero tú te quemaste la espalda
ya, tú, que en la tierra tienes manos de pintor,

ya, claro, y que con la memoria has de abrir la cabeza. 

Así que te sacudiste en la bañera,
yo estoy con mi espejo de agua,

¿de qué manera? ¿Tú que me dijiste que me preguntaste? 

Tú no me lo dijiste, ¿eh? 

Yo te dije que me disculpaste
¿para qué? Ya lo sabes. 

Ya, claro, ya. ¡No! 

No, no, no me lo dijiste. No me lo dijiste. 

Ni siquiera me dijiste una palabra,
ni me respondiste una sola vez,

ni me has contestado nada. 
<|endoftext|>


In [14]:
prueba = """
Es porque aún somos pequeños.

Tú no dejabas de mirar por la ventana,
como si las golondrinas fueran a bailar por ti.
Irradiabas el aura y la luz de la vida
la mirada de una niña que ha visto más de lo que quería
por esa ventana que mostraba en el atardecer un nuevo alba.
"""

In [15]:
text_generation(prueba, temperature=0.9, top_p=0.8, entry_length=120)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [02:09<00:00, 129.53s/it]


Es porque aún somos pequeños.

Tú no dejabas de mirar por la ventana,
como si las golondrinas fueran a bailar por ti.
Irradiabas el aura y la luz de la vida
la mirada de una niña que ha visto más de lo que quería
por esa ventana que mostraba en el atardecer un nuevo alba.
¿Entiendes ahora por qué has venido a verte? 

Porque yo soy la mirada de la que ha visto lo que ha visto. 

¿Qué te ha ocurrido? 

Que ya es tarde. 

Pero tú has venido a verte. 

Tú no has venido a verme. 

¿Qué quieres decir? 

No me has venido a mí. 

No me has venido a mí. 

No me has venido a mí. 

Es que no me has venido a mí. 

Pero sí te has venido a mí. 

No me<|endoftext|>


In [16]:
prueba = """El tren repiqueteaba,
andaba lento, suave,
meciendo a los pasajeros
que parados, veían viajar al paisaje
"""
text_generation(prueba, temperature=1., top_p=0.8, entry_length=90)


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:03<00:00, 63.35s/it]

El tren repiqueteaba,
andaba lento, suave,
meciendo a los pasajeros
que parados, veían viajar al paisaje
y cómo navegaban,

sus largos dedos hacían el camino,
delante de los trajes de vuelo
hacia la ventana
de una foto sin fecha, en un ambiente
con calma
y con la verdad se veía cómo llegaba
el tren, nos ponía a la carretera,
los ferrocarriles iban hasta la puerta
de las casas, mientras los pasajeros murmuraban
que nuestra infancia estaba vacía

y la conciencia volvía. 
<|endoftext|>


In [17]:
text_generation(prueba, temperature=1., top_p=0.8, entry_length=90)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:00<00:00, 60.50s/it]

El tren repiqueteaba,
andaba lento, suave,
meciendo a los pasajeros
que parados, veían viajar al paisaje
que penetraba
comenzaba a hacerlo
con una sensación de no haber logrado ver el día
que no era el
697, porque en cada parada se había
tenido la situación en
derruido; y era ese frío
que en el tiempo, en el espacio,
ha herido los nervios y ha desaparecido el miedo. 

Mis ojos se fijaban en el paisaje:
el paisaje tenía una belleza diferente
por<|endoftext|>


In [18]:
text_generation(prueba, temperature=1., top_p=0.8, entry_length=90)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:00<00:00, 60.71s/it]

El tren repiqueteaba,
andaba lento, suave,
meciendo a los pasajeros
que parados, veían viajar al paisaje
y en el parabrisas
el relente que producía el relente
en el morro de un árbol. 

En los cruces de la Plaza de España,
Neptuno como el ave que juguetea,
escribiendo la pintura en azul
y el papel de un niño pequeño
de rodillas, de rodillas,
nuestros pies parecen enfermos
la cabeza está sobre la cama,
la vista en el espejo
al<|endoftext|>


In [19]:
text_generation(prueba, temperature=1., top_p=0.8, entry_length=90)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:02<00:00, 62.87s/it]

El tren repiqueteaba,
andaba lento, suave,
meciendo a los pasajeros
que parados, veían viajar al paisaje
que, como niños, iban en la misma dirección. 

Ahogado en el coche, en el cielo,
crece un mundo, una blancura extraña. 

Helo aquí, en este mundo, es una raza. 

No puedo detener el tiempo, pero este tren me aguarda. 

Lo noto
en el olor. En el aire,
en la luz, entre el cielo. 

Gema se estrem<|endoftext|>


In [20]:
text_generation(prueba, temperature=1., top_p=0.8, entry_length=90)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:00<00:00, 60.93s/it]

El tren repiqueteaba,
andaba lento, suave,
meciendo a los pasajeros
que parados, veían viajar al paisaje
A los pasajeros y camareros
y unos muchachos con voz
realmente infantil. 

Al alejarse, el vagón crujía
Me decía: 

—¡Baje! —(Y si no se trata de un himno, lo hacen). 

—Bueno —dijo el conductor—, voy a ver si tienen un ataúd
A quién se lo pide. 

—No, si no, vamos a buscarlo. 

Todos, al mismo<|endoftext|>


In [21]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_auto_class',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_can_retrieve_inputs_from_name',
 '_convert_head_mask_to_5d',
 '_create_or_get_repo',
 '_expand_inputs_for_generation',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_from_config',
 '_get_backward_hooks',
 '_get_decoder_start_token_id',
 '_get_logits_processor',
 '_get_logits_warper',
 '_get_name',
 '_get_repo_url_from_name',
 '_get_resized_embeddings',
 '_get_resized_lm_head',
 '_get_stopping_crite